# Copies de blueprint amb p i q

In [1]:
import numpy as np

# Asumiendo que ya tienes A2_blueprint (L_original)
L_original = np.load("A2_blueprint.npy")
Nx = L_original.shape[0]

# Parámetros del paper
p = 0.007  # probabilidad de copiar no-arista → convertir a enlace
q = 0.15   # probabilidad de copiar enlace → borrar enlace

# Número de redes sintéticas que quieres generar
K = 4

# Creamos un tensor A de shape (K, Nx, Nx)
A = np.zeros((K, Nx, Nx), dtype=int)

for k in range(K):
    # Empezamos copiando blueprint
    A[k] = L_original.copy()
    # Ahora cada posición (i,j) puede “fallar”:
    for i in range(Nx):
        for j in range(Nx):
            if L_original[i, j] == 1:
                # con probabilidad q lo convertimos a 0
                if np.random.rand() < q:
                    A[k, i, j] = 0
            else:
                # si no hay enlace, con probabilidad p lo convertimos a 1
                if np.random.rand() < p:
                    A[k, i, j] = 1
# Verificar una de las redes generadas
print("Red 0, número de aristas:", A[0].sum())

FileNotFoundError: [Errno 2] No such file or directory: 'A2_blueprint.npy'